In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip /content/drive/MyDrive/WEbmd_pro/Copy-of-webmd.zip

Archive:  /content/drive/MyDrive/WEbmd_pro/Copy-of-webmd.zip
replace webmd.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
data=pd.read_csv('/content/webmd.csv')

In [ ]:
data.columns

In [ ]:
data.isna().sum()

In [ ]:
data1=data[['Reviews','Satisfaction']]

In [ ]:
data1.dropna(inplace=True)

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
#Removing the html strips
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text

In [ ]:
from bs4 import BeautifulSoup
import re

In [ ]:
#Apply function on review column
data1['Reviews'] = data1['Reviews'].apply(denoise_text)

In [ ]:
data1.head()

In [ ]:
#Define function for removing special characters
def remove_special_characters(text, remove_digits=True):
    pattern=r'[^a-zA-z0-9\s]'
    text=re.sub(pattern,'',text)
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"I'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\^^", "", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    return text

In [ ]:
#Apply function on review column
data1['Reviews'] = data1['Reviews'].apply(remove_special_characters)

In [ ]:
#Stemming the text
def simple_stemmer(text):
    ps=nltk.porter.PorterStemmer()
    text= ' '.join([ps.stem(word) for word in text.split()])
    return text

In [ ]:
#Apply function on review column
data1['Reviews']= data1['Reviews'].apply(simple_stemmer)

In [ ]:
data1

In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')

In [ ]:
#set stopwords to english
stop=set(stopwords.words('english'))
print(stop)

In [ ]:
from nltk.tokenize.toktok import ToktokTokenizer

In [ ]:
#Tokenization of text
tokenizer1=ToktokTokenizer()

In [ ]:
#Setting English stopwords
stopword_list=nltk.corpus.stopwords.words('english')

In [ ]:
#removing the stopwords
def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer1.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text

In [ ]:
#Apply function on review column
data1['Reviews'] = data1['Reviews'].apply(remove_stopwords)

In [ ]:
data1['Reviews']

In [ ]:
data1[data1['Satisfaction']>5].index

In [ ]:
data1.drop([195073, 195282, 273577],inplace=True)

In [ ]:
def relabel(z):
  if z<=3:
    return 0
  else:
    return 1
data1['Satisfaction']=data1['Satisfaction'].apply(relabel)

In [ ]:
data1['Satisfaction'].value_counts()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, ENGLISH_STOP_WORDS

In [ ]:
vect = TfidfVectorizer(stop_words=ENGLISH_STOP_WORDS, ngram_range=(1,2), max_features=1000).fit(data1.Reviews)

In [ ]:
X = vect.transform(data1.Reviews)

In [ ]:
X

In [ ]:
Reviews_transformed = pd.DataFrame(X.toarray(),columns=vect.get_feature_names())

In [ ]:
Reviews_transformed

In [ ]:
Reviews_transformed.shape

In [ ]:
x=Reviews_transformed

In [ ]:
y=data1['Satisfaction']

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=143)

In [ ]:
log_model = LogisticRegression()
log_model.fit(X_train,y_train)

In [ ]:
y_pred = log_model.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score,precision_score,r2_score,f1_score

In [ ]:
accuracy_score(y_test,y_pred)

In [ ]:
print('Accuracy is',accuracy_score(y_test,y_pred))
print('precision is',precision_score(y_test,y_pred))
print('r2 is',r2_score(y_test,y_pred))
print('f1 is',f1_score(y_test,y_pred))


In [ ]:
#Random forest classifier
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=200,max_depth=32)
rf.fit(X_train, y_train)

In [ ]:
y=rf.predict(X_test)

In [ ]:
print(accuracy_score(y_test,y)*100)

In [ ]:
print('Accuracy is',accuracy_score(y_test,y_pred))
print('precision is',precision_score(y_test,y_pred))
print('r2 is',r2_score(y_test,y_pred))
print('f1 is',f1_score(y_test,y_pred))
